In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers
import pandas as pd
from datasets import load_dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
!pip install transformers[sentencepiece]
from transformers import Trainer, TrainingArguments

In [3]:
model_checkpoint = "xlm-roberta-base"
from transformers import XLMRobertaTokenizerFast
tokenizer =XLMRobertaTokenizerFast.from_pretrained(model_checkpoint, use_fast=True)
train, test, valid = load_dataset('csv', data_files={'train':'/content/drive/MyDrive/dataset/split_dataset/train.csv','test':'/content/drive/MyDrive/dataset/split_dataset/test.csv','valid':'/content/drive/MyDrive/dataset/split_dataset/valid.csv'}, split=['train','test','valid'])

Using custom data configuration default-5fe555bca5c5b7cb


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5fe555bca5c5b7cb/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [4]:
train

Dataset({
    features: ['Unnamed: 0', 'labels', 'tweets'],
    num_rows: 115000
})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tweets"],truncation=True, padding=True)
    
encoded_train = train.map(tokenize_function, batched=True,num_proc=4)
encoded_test = test.map(tokenize_function, batched=True,num_proc=4)
encoded_valid = valid.map(tokenize_function, batched=True,num_proc=4)

In [52]:
# encoded_dataset
# encoded_dataset=encoded_dataset.remove_columns('tweets')

In [6]:
import torch
new_model=torch.load('/content/drive/MyDrive/dataset/model.pt',map_location='cpu')

In [ ]:
new_model

In [ ]:
# from torch import nn
# modules=list(new_model.children())[:-1]
# new2=nn.Sequential(*modules)
# new2

In [56]:
# class MyModel(nn.Module):
#     def __init__(self, in_features, nb_classes, 
#         hidden_size):
      

#         super(MyModel, self).__init__()
        
#         self.fc1 = nn.Linear(in_features, hidden_size)
#         self.relu=nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, nb_classes)
#         self.sig=nn.Sigmoid()
        
#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x=  self.sig(self.fc2(x))
#         return x

In [57]:
# final=MyModel(768,1,64)

In [ ]:
# new2.add_module(name="final_layer",module=final)
# new2

In [65]:
encoded_train

Dataset({
    features: ['Unnamed: 0', 'attention_mask', 'input_ids', 'labels', 'tweets'],
    num_rows: 115000
})

In [19]:
# from transformers import XLMRobertaForSequenceClassification
# model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')

In [20]:
# from transformers import XLMRobertaForCausalLM
# model = XLMRobertaForCausalLM.from_pretrained('xlm-roberta-base')

In [12]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=5.0,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
    # load_best_model_at_end=True,
)

In [17]:
trainer = Trainer(
    model=new_model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid,
    tokenizer=tokenizer
)

In [18]:
trainer.train()

IndexError: ignored